# Chapter 1: Initial Query with ChatGPT

Dear user, you are part of a consulting firm tasked to: \
[1] identify opportunities for design improvements for a specific product \
[2] conduct benchmarking with competitors whenever appropriate.

In this initial stage, we will initialize the **product** \
and gather initial insights and directions via querying a LLM (**ChatGPT**).

### TO DO SECTION

In [ ]:
'''
Dear user, enter your Product here!
'''

product = "PICO 4 All-in-One VR Headset"

In [ ]:
""" Use this for a fresh set up (Comment if not needed) """
# import os
# import shutil

# if os.path.exists('support'):                     # Delete 'support' folder if it exists
#     shutil.rmtree('support')

### RUN AS INTENDED (DO NOT CHANGE ANYTHING.)

##### Set Up

In [ ]:
! pip install python-dotenv

In [ ]:
""" Set up OpenAI API key """
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")

In [ ]:
search_terms = product

In [ ]:
""" Create directory """
import os

try:                                              # Create directory named after search terms
    os.makedirs("support/%s" % search_terms)
    print("Directory", search_terms, "created")
except FileExistsError:
    print("Directory", search_terms, "exists")

try:                                              # Create directory to store ChatGPT
    os.makedirs("support/%s/%s" % (search_terms, "ChatGPT"))
    print("Directory ChatGPT created")
except FileExistsError:
    print("Directory ChatGPT exists")

##### DESIGN OPPORTUNITIES QUERY with ChatGPT

In [ ]:
""" Design Opportunities Query with ChatGPT """
import pickle
import pandas as pd
import json

from openai import OpenAI
client = OpenAI()

question =  f"Identify and rank ten opportunities for design improvements with the {product}."
template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as a physical component (do not add numbering) and the value as a concised explanation with a maximum of 45 characters. If you don't know the answer to the question, strictly state 'I don't know'."
prompt = question + template

chat_completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt,}],
    temperature=0.5,                                        # adjust persona and temperature
)

result = chat_completion.choices[0].message.content

pickle.dump(result, open("support/%s/%s/chatgpt_design_opportunities.pkl" % (product, "ChatGPT"), "wb"))

data_string = pd.read_pickle("support/%s/%s/chatgpt_design_opportunities.pkl" % (product, "ChatGPT"))
data = json.loads(data_string)

try:
    components = list(data.keys())
    descriptions = list(data.values())

    df = pd.DataFrame({'Component': components, 'Description': descriptions})

    print("Design Opportunities Query with ChatGPT:")
    print(df)
except:
        print("Invalid Response. Please regenerate.")


##### BENCHMARKING QUERY with ChatGPT

In [ ]:
""" BENCHMARKING QUERY with ChatGPT """

""" Get 3 competitors """
import pickle
import pandas as pd
import json

search_terms = product

from openai import OpenAI
client = OpenAI()

question =  f"Identify three competitors of the {product} and the top ten physical components of each competitor."
template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as the product name (do not add numbering) and the value as a Python list of the ten components. If you don't know the answer to the question, strictly state ' '."
prompt = question + template

chat_completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt,}],
    temperature=0.5,                                        # adjust persona and temperature
)

result = chat_completion.choices[0].message.content

pickle.dump(result, open("support/%s/%s/chatgpt_competitors.pkl" % (product, "ChatGPT"), "wb"))

""" Benchmarking """
print("Benchmarking Query with ChatGPT:")
data_string = pd.read_pickle("support/%s/%s/chatgpt_competitors.pkl" % (product, "ChatGPT"))
data = json.loads(data_string)
try:
    competitors = list(data.keys())
    components = list(data.values())
    client = OpenAI()

    for i in range(len(competitors)):
        question = f"Using the {product}, conduct benchmarking with {competitors[i]} with regards to these ten physical components: {data[competitors[i]]}."
        template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as a physical component (do not add numbering) and the value as a concised explanation with a maximum of 45 characters. If you don't know the answer to the question, strictly state 'I don't know.'."
        prompt = question + template

        chat_completion = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,  # adjust persona and temperature
        )

        result = chat_completion.choices[0].message.content

        pickle.dump(result, open("support/%s/%s/chatgpt_%s_benchmarking.pkl" % (product, "ChatGPT", competitors[i]), "wb"))

        data_string = pd.read_pickle("support/%s/%s/chatgpt_%s_benchmarking.pkl" % (product, "ChatGPT", competitors[i]))
        benchmarking_data = json.loads(data_string)
        try:
            components = list(benchmarking_data.keys())
            descriptions = list(benchmarking_data.values())

            df = pd.DataFrame({'Component': components, 'Benchmarking': descriptions})

            print(competitors[i])
            print(df)
            print("\n")
        except:
            print("Invalid Response. Please regenerate.")
except:
        print("Invalid Response. Please regenerate.")